In [ ]:
# #TODO
# Dataset side
# * add apropirate settings for TFRecordDataset -V
# * cache (before transformations) just to avoid disk access (should be ~9gigs) -V
# * move cache to ramfs -V
# * add augmentation ,after cache, probably before transformations 
# * add prefetch, shuffle -V 
# * create validation dataset -V

# Compilation side
# * callbacks:
#     *tensorboard -V
#     *saving weights every epoch -V
#     *learning rate -V
#     *
# * add tensorboard callback -V

# * add hyper parameters (learning rate, learning rate decay)
# * Figure out metrics (accuracy)
#     *add recall ,mean absoulte error, and 'island' recall error -V
#     *sort metrics by stage -V
# * add validation -V

# All
# * add comments

# TPUs
# * try with TPUs -VVV

# *Project
# add augmentation
# add regularization (dropout)

# Dangling layers
# #solve that issue -V

# Results
# *caching seems to eat only gpu memory
# *Intersting to try a ramfs or a BytesIO object , (ramfs is probably better though as tf handles the interface)
# *This should allow to increase batch size
# -Full epoc currently is 1:15h
# TPUs
# -After figuring out TPUs, 13min epoch is achieved!

# -execution adjustments, batch siez, caching.
# -

# Need to figure out what's going on with the PAF stages, all losses seem the same.
# 1.maybe the same loss function is applied to all.
# check names of stages, and maybe have different losses for each.
# check model outputs and graphs
# 2.maybe the dataset is all zeros for pafs, or the limbs are too small,
# optionally disable feedforward links
# 3.another option is that it's ok

# observations
# *when the gradients explode, the PAF stages losses are very different:
# loss: 6034892169562175.0000 - stage1paf_output_loss: 2.7209 - stage2paf_output_loss: 3066.9797 - stage3paf_output_loss: 1367401.7500 - stage4paf_output_loss: 648865408.0000

# explosing gradients seem to have something with reloading checkpoints.
# if true I should open an issue, on econd look it might just be the learning rate.

# Learning rate
# 0.0001 works fine 
# 0.003 explodes
# 0.002 is too much as well


# After training, the PAF stages dont seem to train at all. the converge to a zero minima.\n",
# to solve the issue two solutions are implmented.\n",
# 1.Implement the masking feature in the loss function, for the data in COCO dataset\n",
# 2.Instead of using the paper's approach of hard areas for the PAFs, using a gaussian distribution for the lengths of the joints.\n",
# 3.Adding a start and end gaussian distrubtion to the joint vectors (the beginning and end of the joints), this should improve handling of smaller joints like nose-eye and eye-ear.\n",
# 4.Adding a neck kpt and changing the nose connection to this point. this reflection of anatomy should work better."


# ERROR,
# loss is NaN after 5 batch (128) training.
# -lowering the LR didn't help
# -switching to batch size one to find if is related to data.
# -running 1 batch size, the training is going on fine, before crashing.
# -error occurs at item 815., disabled shuffle
# -retry, happens at 795
# -interesting bug, 

## Enviorment

In [1]:
#verify user
!whoami
#verify user account, if running on google cloud, otherwise ignore
!gcloud auth list
#which enciorment/virtualenv running in
import sys
import os

print("Python running from:",sys.prefix)
print("Current working dir",os.getcwd())

michael_zl_prime
                  Credentialed Accounts
ACTIVE  ACCOUNT
        312164605303-compute@developer.gserviceaccount.com
*       michael.zl.prime@gmail.com

To set the active account, run:
    $ gcloud config set account `ACCOUNT`

Python running from: /usr
Current working dir /home/michael_zl_prime/project_drive/project


In [2]:
#start tensor board
# must run 
#/usr/local/bin/tensorboard serve --logdir gs://dl_training_results/tensorboard --port 8889 --bind_all

---

In [3]:
from config import *

In [4]:
import logging

# get TF logger
log = logging.getLogger('tensorflow')
log.setLevel(logging.DEBUG)

# create formatter and add it to the handlers
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# create file handler which logs even debug messages
fh = logging.FileHandler('/tmp/tensorflow.log')
fh.setLevel(logging.DEBUG)
fh.setFormatter(formatter)
log.addHandler(fh)

In [5]:
import tensorflow as tf
print("Tensorflow version:",tf.version.VERSION)

Tensorflow version: 2.1.0-dev20191203


In [6]:
if TPU_MODE:
    print("Testing results bucket connectivity")
    !touch /tmp/test
    !gsutil cp /tmp/test {TENSORBOARD_PATH}/test
    !gsutil rm {TENSORBOARD_PATH}/test
    !gsutil cp /tmp/test {CHECKPOINTS_PATH}/test
    !gsutil rm {CHECKPOINTS_PATH}/test
    print("Testing dataset bucket connectivity")
    !gsutil ls gs://{GCS_TFRECORDS_BUCKETNAME} | head -4
    print("Testing TPU connectivity")
    !nmap -Pn -p8470 {TPU_IP}
    import tpu_training.init_tpu as init_tpu
    strategy,resolver=init_tpu.init_tpu() #This must be run before any imports!!!!

Testing results bucket connectivity
Copying file:///tmp/test [Content-Type=application/octet-stream]...
/ [1 files][    0.0 B/    0.0 B]                                                
Operation completed over 1 objects.                                              
Removing gs://dl_training_results/tensorboard//test...
/ [1 objects]                                                                   
Operation completed over 1 objects.                                              
Copying file:///tmp/test [Content-Type=application/octet-stream]...
/ [1 files][    0.0 B/    0.0 B]                                                
Operation completed over 1 objects.                                              
Removing gs://dl_training_results/checkpoints//test...
/ [1 objects]                                                                   
Operation completed over 1 objects.                                              
Testing dataset bucket connectivity
gs://datasets_bucket_a/trainin

INFO:tensorflow:Initializing the TPU system: grpc://10.0.3.2:8470


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Clearing out eager caches


INFO:tensorflow:Finished initializing TPU system.


...

INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:7, TPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU_SYSTEM:0, TPU_SYSTEM, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:XLA_CPU:0, XLA_CPU, 0, 0)


In [7]:
import ipywidgets as widgets
import datetime
import dataset_functions
from models.six_stage_linear_model import ModelMaker
import callbacks
import dataset_builder
import load_weights
import loss_metrics

---
# Make dataset

In [8]:
tfrecord_files_train,tfrecord_files_val=dataset_builder.get_tfrecord_filenames()
print("Found the following TFrecords:\n","\n".join(tfrecord_files_train),"\n", "\n".join(tfrecord_files_val))

print("Building training dataset")
dst=dataset_builder.build_training_ds(tfrecord_files_train)
print("Training dataset shape:",dst)
print("Building validation dataset")
dsv=dataset_builder.build_validation_ds(tfrecord_files_val)
print("Validation dataset shape:",dsv)

Retrieving TFrecords in TPU_mode
Found the following TFrecords:
 gs://datasets_bucket_a/training-001.tfrecords
gs://datasets_bucket_a/training-002.tfrecords
gs://datasets_bucket_a/training-003.tfrecords
gs://datasets_bucket_a/training-004.tfrecords
gs://datasets_bucket_a/training-005.tfrecords
gs://datasets_bucket_a/training-006.tfrecords
gs://datasets_bucket_a/training-007.tfrecords
gs://datasets_bucket_a/training-008.tfrecords
gs://datasets_bucket_a/training-009.tfrecords
gs://datasets_bucket_a/training-010.tfrecords
gs://datasets_bucket_a/training-011.tfrecords
gs://datasets_bucket_a/training-012.tfrecords
gs://datasets_bucket_a/training-013.tfrecords
gs://datasets_bucket_a/training-014.tfrecords
gs://datasets_bucket_a/training-015.tfrecords
gs://datasets_bucket_a/training-016.tfrecords
gs://datasets_bucket_a/training-017.tfrecords
gs://datasets_bucket_a/training-018.tfrecords
gs://datasets_bucket_a/training-019.tfrecords
gs://datasets_bucket_a/training-020.tfrecords
gs://datasets_b

In [ ]:
#Test element
# dst_iter=iter(dst)
# sample_elem=next(dst_iter)
# print("Dataset shape:",dst) #this should match the model input, and output stages

In [ ]:
#testing mask
# sample_elem=next(dst_iter)
# m=sample_elem[1][1][0,...,0]
# import matplotlib.pyplot as plt
# plt.imshow(m)
# plt.colorbar()

---
# Callbacks

In [9]:
fit_callbacks=[
    callbacks.checkpoints_callback
    ,callbacks.tensorboard_callback
    ,callbacks.learning_rate_scheduler_callback
    ,callbacks.print_lr_callback
    ,tf.keras.callbacks.TerminateOnNaN()
]


In [ ]:
#run to clean tensorboard results
#!gsutil -m rm -r {TENSORBOARD_PATH}/*
#!gsutil -m rm -r {CHECKPOINTS_PATH}/*



---
# Model
## Load weights

In [10]:
checkpoint=None
starting_epoch=0
if ASK_FOR_CHECKPOINTS:
    checkpoint,starting_epoch=load_weights.checkpoints_prompt()

In [11]:
if INCLUDE_MASK:
    loss=loss_metrics.MaskedMeanSquaredError
    loss_name="Masked_MSE"
else:
    loss=tf.keras.losses.MeanSquaredError
    loss_name="Regular_MSE"

In [12]:
model_maker=ModelMaker() #must be outside scope to keep the graph clean
tf.keras.backend.clear_session() #to clean to backaend from the imported model
def define():
    train_model,test_model=model_maker.create_models()
    
    #this must match the model output order
    metrics=[
          [tf.keras.metrics.MeanAbsoluteError()]
         ,[tf.keras.metrics.MeanAbsoluteError()]
         ,[tf.keras.metrics.MeanAbsoluteError()]
         ,[tf.keras.metrics.MeanAbsoluteError()]
         ,[#tf.keras.metrics.Recall(),
                    tf.keras.metrics.MeanAbsoluteError()]    
         ,[#tf.keras.metrics.Recall(),
                    tf.keras.metrics.MeanAbsoluteError()]
        ]
    
    train_model.compile(
                    optimizer=tf.keras.optimizers.Adam(BASE_LEARNING_RATE)                   
                    ,loss=loss(name=loss_name)
                    ,metrics=metrics                           
                   )
    return train_model,test_model

if TPU_MODE:
    with strategy.scope():
        train_model,test_model=define()
        if (checkpoint):
            train_model.load_weights(checkpoint)
else:
    train_model,test_model=define()
    if (checkpoint):
        train_model.load_weights(checkpoint)

---
Actually training

In [ ]:
train_history=train_model.fit(
    dst
    ,epochs=50
    ,steps_per_epoch=STEPS_PER_EPOCH
    ,validation_data=dsv
    ,callbacks=fit_callbacks
    ,initial_epoch=starting_epoch
)

Train for 437 steps

Learning rate for epoch 0 is 0.00019999999494757503
Epoch 1/50
  7/437 [..............................] - ETA: 31:34 - loss: 0.0843 - stage1paf_output_mask_loss: 0.0076 - stage2paf_output_mask_loss: 0.0078 - stage3paf_output_mask_loss: 0.0078 - stage4paf_output_mask_loss: 0.0079 - stage5heatmap_output_mask_loss: 0.0270 - stage6heatmap_output_mask_loss: 0.0263 - stage1paf_output_mask_mean_absolute_error: 0.0188 - stage2paf_output_mask_mean_absolute_error: 0.0176 - stage3paf_output_mask_mean_absolute_error: 0.0179 - stage4paf_output_mask_mean_absolute_error: 0.0175 - stage5heatmap_output_mask_mean_absolute_error: 0.0793 - stage6heatmap_output_mask_mean_absolute_error: 0.0790

In [ ]:
model_path=MODELS_PATH+datetime.datetime.now().strftime("%d(%a)%m%y-%H%M")
tf.keras.models.save_model(train_model,model_path)

In [ ]:
#shut down TPU
!gcloud compute tpus stop node-1 --zone us-central1-c

In [ ]:
#v.show_pafs_kpts_img(img.numpy(),paf.numpy(),kpt.numpy(),1,1) #can be used to draw the tensor data